In [19]:
import sqlalchemy as sq
import geopandas as gpd  # type: ignore
import pandas as pd  # type: ignore
from dotenv import load_dotenv
import os, sys, calendar

sys.path.append("../")
from Shared.DataService import DataService  # type: ignore

In [15]:
TABLENAME = "agg_station_dly"

load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [17]:
db = DataService(PG_DB, PG_ADDR, PG_PORT, PG_USER, PG_PW)
conn = db.connect()

In [8]:
MIN_MONTH = 1
MAX_MONTH = 12

MIN_YEAR = 1995
MAX_YEAR = 2023

years = [year for year in range(MIN_YEAR, MAX_YEAR + 1)]  # the year range we want to pull data from
months = [month for month in range(MIN_MONTH, MAX_MONTH + 1)]  # the month range we want to pull data from


columns = ['year', 'district']
dates = []
for year in years:
    for month in months:
        numDays = calendar.monthrange(int(year), int(month))[1]

        for day in range(1, numDays + 1):
            dates.append(f'{year}/{month}/{day}')
            columns.append(f'{year}/{month}/{day}:max_temp')
            columns.append(f'{year}/{month}/{day}:min_temp')
            columns.append(f'{year}/{month}/{day}:mean_temp')
            columns.append(f'{year}/{month}/{day}:total_rain')
            columns.append(f'{year}/{month}/{day}:total_snow')
            columns.append(f'{year}/{month}/{day}:total_precip')
            columns.append(f'{year}/{month}/{day}:snow_on_grnd')

In [12]:

df = pd.DataFrame(columns=columns)

query = sq.text(
    f"""
    SELECT AVG(max_temp) AS max_temp, AVG(min_temp) AS min_temp, AVG(mean_temp) AS mean_temp, AVG(total_rain) AS total_rain, 
        AVG(total_snow) AS total_snow, AVG(total_precip) as total_precip, AVG(snow_on_grnd) as snow_on_grnd 
	FROM public.{ab_station_data}
	JOIN public.stations_dly ON stations_dly.station_id = {ab_station_data}.station_id
	WHERE stations_dly.district = {4840} AND {ab_station_data}.date = '{1995/2/01}'
    """
)

NameError: name 'sq' is not defined

In [10]:
df

,year,district,1995/1/1:max_temp,1995/1/1:min_temp,1995/1/1:mean_temp,1995/1/1:total_rain,1995/1/1:total_snow,1995/1/1:total_precip,1995/1/1:snow_on_grnd,1995/1/2:max_temp,...,2023/12/30:total_snow,2023/12/30:total_precip,2023/12/30:snow_on_grnd,2023/12/31:max_temp,2023/12/31:min_temp,2023/12/31:mean_temp,2023/12/31:total_rain,2023/12/31:total_snow,2023/12/31:total_precip,2023/12/31:snow_on_grnd


In [39]:
idQuery = sq.text("SELECT car_uid FROM public.census_ag_regions")
uids = pd.read_sql_query(idQuery, conn)

In [40]:
mbIDs = [uid for uid in uids['car_uid'] if uid < 4700]
skIDs = [uid for uid in uids['car_uid'] if uid >= 4700 and uid < 4800]
abIDs = [uid for uid in uids['car_uid'] if uid >= 4800]

In [ ]:
for year in years:
    # query all districts for mb
        # query for each day
    # query all districts for sk
        # query for each day
    # query all districts for ab
        # query for each day

    # merge all of these into the dataframe